In [ ]:
import os
import re
import sys
import glob
import pickle
import tables
from collections import OrderedDict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from comet_ml.api import API, APIExperiment
from comet_ml.query import Tag

if '..' not in sys.path:
    sys.path.append('..')
from deep_utils import *
import pypan.utils

#### Find the best experiment given a set of tags

In [ ]:
api = API(api_key = os.environ['COMET_API_KEY'])
workspace = 'danielelinaro'
project_name = 'inertia'
D = 2
DZA = 60
network_name = 'IEEE39'
inertia_units = 'GW s'
area_ID = 1
bus_IDs = [3, 14, 17]
# inertia of generator 1
H_G1 = 25 # [s]

query = Tag(network_name) & \
        Tag(f'D={D}') & \
        Tag(f'DZA={DZA}') & \
        Tag('1D_pipeline') & \
        Tag('buses_' + '-'.join(map(str,bus_IDs))) & \
        Tag(f'H_G1_{H_G1}') & \
        Tag(f'area{area_ID}')

experiments = api.query(workspace, project_name, query, archived=False)
experiment_IDs = []
MAPE = []
val_loss = []
tags =  []
for experiment in experiments:
    ID = experiment.id
    experiment_IDs.append(ID)
    sys.stdout.write(f'Downloading data for experiment ID {ID}... ')
    metrics = experiment.get_metrics()
    sys.stdout.write('done.\n')
    val_loss.append(np.array([float(m['metricValue']) for m in metrics if m['metricName'] == 'val_loss']))
    has_MAPE = False
    for m in metrics:
        if m['metricName'] == 'mape_prediction':
            val = m['metricValue']
            try:
                MAPE.append(float(val))
            except:
                MAPE.append(list(map(float, [v for v in val[1:-1].split(' ') if len(v)])))
            has_MAPE = True
            break
    tags.append(experiment.get_tags())
    print(f'  val_loss: {val_loss[-1].min():.4f}')
    if has_MAPE:
        print(f'      MAPE: {MAPE[-1]}%')
    else:
        print('      MAPE: [experiment not terminated]')
    print('      Tags: "{}"'.format('" "'.join(tags[-1])))
# idx = np.argmin(MAPE)
idx = np.argmin([loss.min() for loss in val_loss])
experiment_ID = experiment_IDs[idx]
MAPE = MAPE[idx]
val_loss = val_loss[idx]
tags = tags[idx]
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE}%).')

In [ ]:
experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(experiments_path + experiment_ID \
                                      + '/parameters.pkl', 'rb'))
epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
model = keras.models.load_model(best_checkpoint, compile=True)
data_dirs = ['..' + os.path.sep + os.path.sep.join([d for d in data_dir.split(os.path.sep) if '{}' not in d]) \
             for data_dir in network_parameters['data_dirs']]
# we need mean and standard deviation of the training set to normalize the data
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
data_dir = data_dirs[0]
if not os.path.isdir(data_dir):
    raise Exception(f'{data_dir}: no such directory')
print(f'Loaded network from {best_checkpoint}.')
print(f'Data directory is {data_dir}.')

In [ ]:
keras.utils.plot_model(model, show_shapes=False, dpi=96)

## Constant inertia
1. the value of inertia of area 1 is either 5.405 or 6.0 GW s
1. **omega** and **Pe** of the three areas are used for the estimation

In [ ]:
default_H = OrderedDict([
    ('Pg30', H_G1), ('Pg31', 15.15), ('Pg32', 17.9), ('Pg33', 14.3), ('Pg34', 13),
    ('Pg35', 17.4), ('Pg36', 13.2), ('Pg37', 12.15), ('Pg38', 17.25), ('Pg39', 21)

])
P_nom = {gen_ID: 100e6 for gen_ID in default_H}

generators_areas_map = [
    ['Pg31', 'Pg32', 'Pg39']
]

window_dur = 60
window_step = 10

H_values = [
    default_H,
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 17), ('Pg32', 20), ('Pg33', 14.3), ('Pg34', 13),
        ('Pg35', 17.4), ('Pg36', 13.2), ('Pg37', 12.15), ('Pg38', 17.25), ('Pg39', 23)

    ])
]
N_H = len(H_values)

area_inertia = []

data_normalized = []
inertia = []

var_names = [var_name.format(bus_ID) for bus_ID in np.unique(bus_IDs) for var_name in ['omegael_bus{}', 'Pe_bus{}']]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

for H in H_values:
    tmp_inertia = []
    for generator_IDs in generators_areas_map:
        tmp_inertia.append(np.sum([P_nom[gen_ID] * H[gen_ID] for gen_ID in generator_IDs]) * 1e-9)
    area_inertia.append(tmp_inertia)
    data_file = data_dir + '/ieee39_' + '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'

    t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        data_mean,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        verbose = True)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, HH, _ = predict(model, data_sliding, window_step, dt)
    inertia.append(HH)

In [ ]:
fig = plt.figure(figsize=(10, 3 * 3))
gs = fig.add_gridspec(3, 4)
ax = []
ax.append([fig.add_subplot(gs[0, :3]), fig.add_subplot(gs[0, 3])]),
ax.append([fig.add_subplot(gs[1, :3]), fig.add_subplot(gs[1, 3])]),
ax.append([fig.add_subplot(gs[2, :2]), fig.add_subplot(gs[2, 2:])])

col = [[.2,.2,.2], [.8,0,0], [0,.7,0]]

var_names = 'omegael_bus{}','Pe_bus{}'
bus_ID = 14

idx = t < 180
for i in range(N_H):
    for j,var_name in enumerate(var_names):
        key = var_name.format(bus_ID)
        value = data_normalized[i][key]
        n,edges = np.histogram(value, bins=25, range=(-4,4), density=True)
        ax[j][0].plot(t[idx], value[idx], color=col[i], lw=1, label=f'H = {area_inertia[i][0]} {inertia_units}')
        ax[j][1].plot(n, edges[:-1] + np.diff(edges[:2])[0] / 2, color=col[i], lw=1)
        for a in ax[j]:
            a.set_ylim([-4,4])
    ax[-1][i].plot(time / 60, inertia[i], 'k', lw=1)
    ax[-1][i].plot(time[[0,-1]] / 60, area_inertia[i][0] + np.zeros(2), 'k--', lw=1)
    ax[-1][i].set_ylim([5, 6.5])
    ax[-1][i].set_xlabel('Time [min]')

for a in ax:
    for side in 'right','top':
        for i in range(2):
            a[i].spines[side].set_visible(False)

ax[0][0].legend(loc='best')

for i in range(0, len(ax)-1, 2):
    ax[i][0].set_ylabel(r'$\omega_{e}$')
    ax[i+1][0].set_ylabel(r'$P_{e}$')
ax[-1][0].get_shared_x_axes().join(ax[-1][0], ax[-1][1])

ax[-1][0].set_xlim([0,30])
ax[-1][0].set_ylabel(f'Inertia [{inertia_units}]')
ax[-2][0].set_xlabel('Time [s]')
ax[-2][1].set_xlabel('Fraction')
fig.tight_layout()
output_filename = f'{network_name}_area{area_ID}_H_G1={H_G1}_buses={"-".join(map(str,bus_IDs))}_const_H.pdf'
fig.savefig(output_filename)

## Steps of inertia with transient
1. the inertia of area 1 steps from 6.5 to 5.5 at 20 s
1. the inertia of area 2 steps from 6.175 to 6.5 at 40 s
1. **omega** and **Pe** of the both areas used for the estimation
1. one single simulation

In [ ]:
H_values = [
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 10), ('Pg32', 14), ('Pg33', 14.3), ('Pg34', 13),
        ('Pg35', 17.4), ('Pg36', 13.2), ('Pg37', 12.15), ('Pg38', 17.25), ('Pg39', 18)

    ]),
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 21), ('Pg32', 23), ('Pg33', 14.3), ('Pg34', 13),
        ('Pg35', 17.4), ('Pg36', 13.2), ('Pg37', 12.15), ('Pg38', 17.25), ('Pg39', 25)

    ])
]

H = []
for H_val in H_values:
    for generators in generators_areas_map:
        H.append(np.sum([H_val[gen_id] * P_nom[gen_id] for gen_id in generators]) * 1e-9)

data_file = data_dir + '/' + network_name.lower()
for gen_id in H_values[0]:
    data_file += '_' + '-'.join(map(lambda h: f'{h:.3f}', np.unique([H[gen_id] for H in H_values])))
data_file += '.h5'

window_dur = 60
window_step = 10

var_names = [var_name.format(bus_ID) for bus_ID in bus_IDs for var_name in ['omegael_bus{}', 'Pe_bus{}']]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

t, _, data_normalized, data_sliding, _ = load_data_slide([data_file],
                                                         var_names,
                                                         data_mean,
                                                         data_std,
                                                         window_dur,
                                                         window_step,
                                                         verbose = True)

dt = np.diff(t[:2])[0]
time, inertia, _ = predict(model, data_sliding, window_step, dt)

In [ ]:
fig = plt.figure(figsize = (6,9))
gs = fig.add_gridspec(8, 1)
ax = [
    fig.add_subplot(gs[:2,0]),
    fig.add_subplot(gs[2:4,0]),
    fig.add_subplot(gs[4:-1,0]),
    fig.add_subplot(gs[-1,0])
]

ds = 5
for i,bus_id in enumerate(bus_IDs):
    ax[0].plot(t[::ds] / 60, data_normalized[f'omegael_bus{bus_id}'][::ds], color=col[i], lw=1, label=f'bus{bus_id}')
    ax[1].plot(t[::ds] / 60, data_normalized[f'Pe_bus{bus_id}'][::ds], color=col[i], lw=1)
ax[0].set_ylim([-5,5])
ax[1].set_ylim([-5,5])

ylim = [H[0] - 0.5, H[1] + 0.5]
ax[2].plot(time[::ds] / 60, inertia[::ds,0], color=col[0])
ax[2].set_ylim(ylim)
ax[2].plot([0, 30], H[0] + np.zeros(2), '--', color=col[0], lw=1)
ax[2].plot([30, 60], H[1] + np.zeros(2), '--', color=col[0], lw=1)

ax[3].plot([0, 30], H[0] + np.zeros(2), '-', color=col[0], lw=2)
ax[3].plot([30, 60], H[1] + np.zeros(2), '-', color=col[0], lw=2)
ax[3].plot(30 + np.zeros(2), ylim, '--', color=col[0], lw=1)
ax[3].set_ylim(ylim)
ax[3].set_yticks([4,5,6,7])
ax[3].set_xlabel('Time [min]')

ax[0].legend(loc='lower right')

for a in ax:
    a.set_xlim([-1,61])
    for side in 'right', 'top':
        a.spines[side].set_visible(False)
    if a != ax[-1]:
        a.set_xticklabels([])

ax[0].set_ylabel(r'$\omega_{e}$')
ax[1].set_ylabel(r'$P_{e}$')
ax[2].set_ylabel(f'Inertia [{inertia_units}]')
ax[3].set_ylabel(f'Inertia [{inertia_units}]')

fig.tight_layout()
output_filename = f'{network_name}_area{area_ID}_H_G1={H_G1}_buses={"-".join(map(str,bus_IDs))}_step_H.pdf'
fig.savefig(output_filename)